In [1]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.7522, 0.7893, 0.1847],
        [0.9452, 0.4827, 0.5630],
        [0.6658, 0.5401, 0.6368],
        [0.3461, 0.2127, 0.1284],
        [0.2034, 0.3083, 0.2471]])


In [2]:
45+7

52